<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Загрузка данных

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.shape

(5000, 5)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
data.duplicated().sum() 

153

In [7]:
#Есть полные дубликаты, удалим их
data = data.drop_duplicates()

In [8]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811842,1.203425,0.152259
std,0.500049,8.487995,9972.953985,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33200.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


В данных нет аномалий и пропусков, полные дубликаты были удалены. 

## Умножение матриц

Отбираем матрицу признаков и зависимую переменную

In [9]:
X = data.drop('Страховые выплаты', axis = 1).values
X

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

In [10]:
y = data['Страховые выплаты'].values
y

array([0, 1, 0, ..., 0, 0, 0])

Создаем квадратную матрицу inv рандомно

In [11]:
inv = np.random.rand(4,4)

In [12]:
# Проверим inv на обратимость
np.linalg.inv(inv)

array([[ 0.04456328, -0.35706131,  1.60606817, -0.86473126],
       [ 1.18786366,  0.44767697, -0.45223811, -0.20776721],
       [-0.8036034 ,  1.42231917, -1.34016815,  0.93529338],
       [-0.26734519, -1.53222954,  1.29618223,  0.60347778]])

In [13]:
# умножаем матрицу признаков на рандомную квадратную матрицу
X_inv = X.dot(inv)

In [14]:
X_inv.shape

(4847, 4)

In [15]:
#Модель без изменения матрицы признаков
model_1 = LinearRegression()
model_1.fit(X,y)
pred_1 = model_1.predict(X)
r1 = r2_score(y, pred_1)
r1

0.4302010044852066

In [16]:
#Модель с измененной матрицей признаков
model_2 = LinearRegression()
model_2.fit(X_inv,y)
pred_2 = model_2.predict(X_inv)
r2 = r2_score(y, pred_2)
r2

0.4302010044852085

In [17]:
print(model_1.intercept_)
print(model_1.coef_)
print(model_2.intercept_)
print(model_2.coef_)

-0.9517099946520766
[ 9.46018141e-03  3.61640081e-02 -1.38991363e-07 -1.42517371e-02]
-0.9517099946548366
[-0.00016749  0.03038831  0.03050516 -0.06654148]


- Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
 - **Не изменится.**
- Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
 - Константа не изменилась (w0), изменились значения w1

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $A$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится. R2 неизменился

**Обоснование:** 
$$
a = Xw
$$
$$
w = (X^T X)^{-1} X^T y
$$

Тогда, если А - рандомно сгенерированная матрица, на которую умножается X (получаем АХ), то подставив формулу предсказания формулу обучения и используя свойства матриц

1) Для квадратных матриц
$$
(AB)^{-1} = B^{-1} A^{-1} 
$$
$$
(ABС)^{-1} = ((AB)С)^{-1} = ((AB)С)^{-1} = С^{-1}(AB)^{-1} = С^{-1}B^{-1}A^{-1}
$$
2) Для любых матриц
$$
(AB)^T=B^T A^T
$$

получаем:

$$
a1 = XA((XA)^T XA)^{-1} (XA)^T y = XA*((A)^T X^T X A)^{-1} * (XA)^T y =
$$

(A)^T (X^T X) A - три квадратные матрицы (4х4), поэтому можем применить к ним свойства квадратных матриц =>

$$
XAA^{-1} *(X^T *X)^{-1} * (A^T)^{-1} *(XA)^T y =
$$

$$
XE *(X^T *X)^{-1} * (A^T)^{-1} * A^T * X^T * y =
$$

$$
X *(X^T *X)^{-1} * E * X^T * y=> 
$$

можем привести это к виду

$$
a1 = X * (X^T X)^{-1} X^T y
$$

получается, что a = a1, предсказания совпадают

**Ответ на вопрос о связи параметров линейной регрессии в исходной задаче и преобразованной**

$$
w = (X^T X)^{-1} X^T y
$$

$$
wp = ((XA)^T XA)^{-1} (XA)^T y
$$

$$
wp/w = (A^{-1} *(X^T *X)^{-1} * (A^T)^{-1} *(XA)^T)/((X^T X)^{-1} X^T)
$$

$$
wp/w = (A^{-1} *(X^T *X)^{-1} * X^T )/((X^T *X)^{-1} * X^T)
$$

$$
w/wp = A
$$

Соответственно, параметры в исходной линейной регрессии и в преобразованной относятся как матрица А (на которую домножались значения исходной матрицы)

## Алгоритм преобразования

**Алгоритм**
- 1) Генерируем рандомную квадратную матрицу K размера (n,n), где n = числу столбцов исходной матрицы признаков А
- 2) Проверяем, что K обратимая матрица (D не равно 0)
- 3) Если К необратимая матрица, генерируем новую
- 4) Создаем матрицу P = KA
- 5) Обучаем модель на матрице признаков P

**Обоснование**

Умножение матрицы признаков на рандомную обратимую матрицу не меняет качество модели линейной регрессии, исходя из обоснования из пункта 2.

In [18]:
def hide_features(matrix): #на вход подается матрица признаков
    n = matrix.shape[1] 
    K = np.random.rand(n,n) 
    while np.linalg.det(K)==0: #если сгенерированная матрица К необратимая, генерируется новая, пока не получится обратимая
        K = np.random.rand(n,n)
    P = matrix.dot(K)
    return P, K #на выходе получаем P - зашифрованная матрица, K - клюяч для ее расшифровки

## Проверка алгоритма

In [19]:
print("Незашифрованные данные",X, sep = '\n')
X_hidden, key = hide_features(X)
print("Зашифрованные данные", X_hidden, sep = '\n')
print("Матрица ключей", key, sep = '\n')

Незашифрованные данные
[[1.00e+00 4.10e+01 4.96e+04 1.00e+00]
 [0.00e+00 4.60e+01 3.80e+04 1.00e+00]
 [0.00e+00 2.90e+01 2.10e+04 0.00e+00]
 ...
 [0.00e+00 2.00e+01 3.39e+04 2.00e+00]
 [1.00e+00 2.20e+01 3.27e+04 3.00e+00]
 [1.00e+00 2.80e+01 4.06e+04 1.00e+00]]
Зашифрованные данные
[[37169.98395227 32441.905747   37126.68225769 47708.45534165]
 [28484.57596592 24863.15659308 28457.82912393 36560.35852841]
 [15743.50341406 13742.23808427 15729.98191577 20206.38359334]
 ...
 [25399.30628958 22168.14240469 25367.41973212 32602.67471717]
 [24502.69177899 21385.472469   24472.57361256 31451.27395954]
 [30422.44378908 26552.0348282  30384.71028549 39048.20341473]]
Матрица ключей
[[0.92961609 0.31637555 0.18391881 0.20456028]
 [0.56772503 0.5955447  0.96451452 0.6531771 ]
 [0.74890664 0.65356987 0.74771481 0.96130674]
 [0.0083883  0.10644438 0.29870371 0.65641118]]


In [20]:
model_3 = LinearRegression()
model_3.fit(X_hidden,y)
pred_3 = model_3.predict(X_hidden)
r3 = r2_score(y, pred_3)
r3

0.4302010044851894

In [21]:
print('Качество регрессионной модели на исходных данных:', r1 )
print('Качество регрессионной модели на зашифрованных данных:', r3 )

Качество регрессионной модели на исходных данных: 0.4302010044852066
Качество регрессионной модели на зашифрованных данных: 0.4302010044851894


## Выводы

- Данные пользователей могут быть зашифрованы при помощи свойств матриц
- В данной работе мы разработали алгоритм, который, получив на вход матрицу признаков (features) преобразовывает ее в новую матрицу, где данные имеют другие значения. Дальнейшая работа с такой преобразованной матрицей позволяет проводить моделирование так же, как это было бы с незашифрованными признаками. При этом, качество модели при обращени к данному методу шифрования, не меняется (R2 не меняется)